<div align="center"><img src="https://i.imgur.com/S9enGUW.png" width="500">
</div>

<div align="center"><img src="https://storage.googleapis.com/kaggle-organizations/3788/thumbnail.png" width="500">
</div>



<div class="alert alert-block alert-warning" align="center">  
   <h1> Some Intels</h1>
</div>

----

<div align="center">
    <img src="https://www.paradisosolutions.com/blog/wp-content/uploads/2020/08/THE-IMPACT-OF-COVID-19-ON-EDUCATION.png">
</div>

* [Impact Of Covid-19 - Education Untape Potential of E-learning](https://www.paradisosolutions.com/blog/impact-covid-19-education-untape-potential-e-learning)


<div align="center">
    <img src="https://blogs.sap.com/wp-content/uploads/2020/04/2020-04-22_12-24-42.png">
</div>



<div class="alert alert-block alert-warning" align="center">  
   <h1> Probleme Statements</h1>
    <div class="alert alert-block alert-warning">  
     The <b>COVID-19 Pandemic</b> has disrupted learning for more than <b>56 million students in the United States</b>. 
     In the Spring of 2020, most states and local governments across the U.S. closed educational institutions to stop the spread of the virus. 
        In response, schools and teachers have attempted to reach students remotely through distance learning tools and digital platforms. Until today, concerns of the exacaberting digital divide and long-term learning loss among America’s most vulnerable learners continue to grow.
    
   
    
</div>

<div class="alert alert-block alert-success" align="center">  
<h1> Challenge</h1>
</div> <br/>      

We challenge the Kaggle community to explore 
- (1) the state of digital learning in 2020 and 
- (2) how the engagement of digital learning relates to factors such as district demographics, broadband access, and state/national level policies and events.

We encourage you to guide the analysis with questions that are related to the themes that are described above (in bold font). Below are some examples of questions that relate to our problem statement:

* What is the picture of digital connectivity and engagement in 2020?
* What is the effect of the COVID-19 pandemic on online and distance learning, and how might this also evolve in the future?
* How does student engagement with different types of education technology change over the course of the pandemic?
* How does student engagement with online learning platforms relate to different geography? Demographic context (e.g., race/ethnicity, ESL, learning disability)? Learning context? Socioeconomic status?
* Do certain state interventions, practices or policies (e.g., stimulus, reopening, eviction moratorium) correlate with the increase or decrease online engagement?

<div align="center">
    <img src="https://i.pinimg.com/originals/79/0f/91/790f91d626b5ee83bc7e55eb8b4807dc.png">
</div>



# Importing Libraries 📚📚📚

In [ ]:
import numpy as np 
import pandas as pd 
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import wandb
import math

import folium
from geopy.geocoders import Nominatim
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

import gc




# Reading data files 👓


----

### Product information data

    
> The Product file ```products_info.csv``` includes information about the characteristics of the top 372 products with most users in 2020. The categories listed in this file are part of LearnPlatform's product taxonomy. 

<div class="alert alert-block alert-info">
📝 Some products may not have labels due to being duplicate, lack of accurate url or other reasons.
  
</div>   
                   
----


| Name                       | Description                                                                                                                                                                                                                                                                                                                    |
|----------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| LP ID                      | The unique identifier of the product                                                                                                                                                                                                                                                                                           |
| URL                        | Web Link to the specific product                                                                                                                                                                                                                                                                                               |
| Product Name               | Name of the specific product                                                                                                                                                                                                                                                                                                   |
| Provider/Company Name      | Name of the product provider                                                                                                                                                                                                                                                                                                   |
| Sector(s)                  | Sector of education where the product is used                                                                                                                                                                                                                                                                                  |
| Primary Essential Function | The basic function of the product. There are two layers of labels here. Products are first labeled as one of these three categories: LC = Learning & Curriculum, CM = Classroom Management, and SDO = School & District Operations. Each of these categories have multiple sub-categories with which the products were labeled |
|                            |                                                              

 

In [ ]:
products_df = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
products_df.head()

In [ ]:
products_df.info()

__Observation:__
<div class="alert alert-block alert-warning">  
 <b>About 372 Entries.</b> and 6 features<br/>
  * LP ID as int64 <br/>
  * URL, Product Name,  Provider/Company Name, Sector(s), Primary Essential Function ; are all Objects<br/>
</div>    


---
---
### District information data


> The district file ```districts_info.csv``` includes information about the **characteristics of school districts**, including data from 
>- NCES (2018-19), 
>- FCC (Dec 2018), and 
>- Edunomics Lab. 
<div class="alert alert-block alert-info">

Steps taken to preserve Privacy 🔒 
    
- Identifiable information about the school districts has been removed. 
- An open source tool ARX (Prasser et al. 2020) was used to transform several data fields and reduce the risks of re-identification. 

📝 For data generalization purposes some data points are released with a range where the actual value falls under. Additionally, there are many missing data marked as 'NaN' indicating that the data was suppressed to maximize anonymization of the dataset.

</div>   

| Name                   | Description                                                                                                                                                                                                                                                                              |
|------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| district_id            | The unique identifier of the school district                                                                                                                                                                                                                                             |
| state                  | The state where the district resides in                                                                                                                                                                                                                                                  |
| locale                 | NCES locale classification that categorizes U.S. territory into four types of areas: City, Suburban, Town, and Rural. See Locale Boundaries User's Manual for more information.                                                                                                          |
| pct_black/hispanic     | Percentage of students in the districts identified as Black or Hispanic based on 2018-19 NCES data                                                                                                                                                                                       |
| pct_free/reduced       | Percentage of students in the districts eligible for free or reduced-price lunch based on 2018-19 NCES data                                                                                                                                                                              |
| countyconnectionsratio | ratio (residential fixed high-speed connections over 200 kbps in at least one direction/households) based on the county level data from FCC From 477 (December 2018 version). See FCC data for more information.                                                                         |
| pptotalraw             | Per-pupil total expenditure (sum of local and federal expenditure) from Edunomics Lab's National Education Resource Database on Schools (NERD$) project. The expenditure data are school-by-school, and we use the median value to represent the expenditure of a given school district. |
                                                         


In [ ]:
districts_df = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
districts_df.head()

In [ ]:
districts_df.info()

__Observation:__
<div class="alert alert-block alert-warning">  
 <b>About 233 Entries.and 7 features</b> <br/>
   * distric_id as type int64 <br/>
   * state, locale, pct_black/hispanic, pct_free/reduced, country_connections_ration, pp_total_raw, all are object type<br/>
   
</div>    

### Engagement data


> The engagement data are aggregated at school district level, and each file in the folder ```engagement_data``` represents data from **one school district***. 
<div class="alert alert-block alert-info">
📝The 4-digit file name represents ```district_id``` which can be used to link to district information in ```district_info.csv```. 

📝The ```lp_id``` can be used to link to product information in ```product_info.csv```.
</div>

| Name             | Description                                                                                                    |
|------------------|----------------------------------------------------------------------------------------------------------------|
| time             | date in "YYYY-MM-DD"                                                                                           |
| lp_id            | The unique identifier of the product                                                                           |
| pct_access       | Percentage of students in the district have at least one page-load event of a given product and on a given day |
| engagement_index | Total page-load events per one thousand students of a given product and on a given day                         |
    

In [ ]:
path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    district_id = filename.split("/")[4].split(".")[0]
    df["district_id"] = district_id
    li.append(df)
    
engagement_df = pd.concat(li)
engagement_df = engagement_df.reset_index(drop=True)
engagement_df.head()

In [ ]:
engagement_df.info()

__Observation:__
<div class="alert alert-block alert-warning">  
 <b>About 22M Entries.</b> and 5 Features<br/>
    * lp_id , pct_access, engagement_index, are type float64<br/>
    * time, and district_id as type object<br/>

</div>    

In [ ]:
#districts_df = reduce_mem_usage(districts_df)
#products_df = reduce_mem_usage(products_df)
#engagement_df = reduce_mem_usage(engagement_df)

In [ ]:
gc.collect()

# Missing values🔮


In [ ]:
msno.bar(products_df,color='#09b7a0', sort="ascending", figsize=(10,5), fontsize=12)
plt.show()

__Observation:__

<div class="alert alert-block alert-danger">  
    
The features with missing values are <b>Sectors(s), Primary Essentail Function</b> with <b>20 NANs</b>
</div>

In [ ]:
msno.bar(districts_df,color='#dbdb14', sort="ascending", figsize=(10,5), fontsize=12)
plt.show()

__Observation:__
<div class="alert alert-block alert-danger">  
All the features have <b>10 to 20%</b> of Missing values
</div>  

In [ ]:
msno.bar(engagement_df,color='#14b7db', sort="ascending", figsize=(10,5), fontsize=12)
plt.show()

In [ ]:
gc.collect()

__Observation:__

<div class="alert alert-block alert-danger">  
<b>Engagement_index</b>  is the Features with most missing values
    
</div>  

# Exploratory Data Analysis 📈📊📉


<div class="alert alert-block alert-success" align="center">  

   <h2>States Distribution by Districts</h2>
    
</div>  

In [ ]:
lst = districts_df.state.value_counts()
vals = districts_df.state.dropna().unique()
df1 = pd.DataFrame(columns=['Count'],index= vals)
df1.Count = lst
df1 = df1.sort_values(by='Count', ascending=False)
### plot bar 
fig = px.bar(df1, x ="Count" , y =df1.index ,title="States Distribution",
              color_continuous_scale ='RdBu',color="Count",height=500)
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
sizes = districts_df.state.value_counts()
labels = districts_df.state.dropna().unique()
df_1 = pd.DataFrame(columns=['Count'],index= labels)
df_1.Count = sizes

### plot pie 
fig = px.pie(df_1, values='Count', names=df_1.index, title='States Distribution')
fig.show()

<div class="alert alert-block alert-success" align="center">  

   <h2>Locals Distribution by Districts</h2>
    
</div>  

In [ ]:
sizes = districts_df.locale.value_counts()
labels = districts_df.locale.dropna().unique()
df2 = pd.DataFrame(columns=['Count'],index= labels)
df2.Count = sizes

### plot pie 
fig = px.pie(df2, values='Count', names=df2.index, title='Locals Distribution')
fig.show()

In [ ]:
lst = districts_df.locale.value_counts()
vals = districts_df.locale.dropna().unique()
df1_1 = pd.DataFrame(columns=['Count'],index= vals)
df1_1.Count = lst
df1_1 = df1_1.sort_values(by='Count', ascending=False)
### plot bar 
fig = px.bar(df1_1, x ="Count" , y =df1_1.index ,title="Locale boundaries Distribution",
              color_continuous_scale ='RdBu',color="Count",height=500)
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
gc.collect()

<div class="alert alert-block alert-success" align="center">  

   <h2> US State By Locale boundaries Schools Districts</h2>
    
</div>  

In [ ]:
state_by_locale = districts_df.groupby("state")["locale"].value_counts().to_frame().rename(columns={"locale": "Schools Districts by Local Bounds."}).reset_index()

fig = px.treemap(state_by_locale, path=['state', 'locale'], values='Schools Districts by Local Bounds.',color='Schools Districts by Local Bounds.', 
                 color_continuous_scale='Picnic',
                title=('States by Local, Number of Schools Districts Distribution'))
fig.update_layout(title_x=0.5)
                
fig.show()

<div class="alert alert-block alert-success" align="center">  

   <h2>Top 5;10;20 Product Providers / Company Name</h2>
    
</div>  

In [ ]:
lst = products_df["Provider/Company Name"].value_counts()
vals = products_df["Provider/Company Name"].dropna().unique()
df3__ = pd.DataFrame(columns=['Count'],index= vals)
df3__.Count = lst
df3__ = df3__.sort_values(by='Count', ascending=False)[:5]
### plot bar 
fig = px.scatter(df3__, x =df3__.index , y ="Count",size="Count", color="Count", title="Top 5 Product Providers")
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
gc.collect()

__Observation:__
<div class="alert alert-block alert-warning">  
 <b>Google LLC</b> 1st with 30. <br>
 <b>Houghton Mifflin Harcourt, AND Microsoft<b> 2nd with 6 each.<br>
 <b>IXL Learning, AND Learning A-Z</b> 3rd with 4 each<br>
</div>    


In [ ]:
lst = products_df["Provider/Company Name"].value_counts()
vals = products_df["Provider/Company Name"].dropna().unique()
df3 = pd.DataFrame(columns=['Count'],index= vals)
df3.Count = lst
df3 = df3.sort_values(by='Count', ascending=False)[:10]
### plot bar 
fig = px.scatter(df3, x =df3.index , y ="Count",size="Count", color="Count", title="Top 10 Product Providers",height=500)
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
lst = products_df["Provider/Company Name"].value_counts()
vals = products_df["Provider/Company Name"].dropna().unique()
df3_ = pd.DataFrame(columns=['Count'],index= vals)
df3_.Count = lst
df3_ = df3_.sort_values(by='Count', ascending=False)[:20]
### plot bar 
fig = px.scatter(df3_, x =df3_.index , y ="Count",size="Count", color="Count", title="Top 20 Product Providers")
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
gc.collect()

<div class="alert alert-block alert-success" align="center">  

   <h2>Sectors Distribution</h2>
    
</div>  

In [ ]:
# Credit goes for @anki112279 
count = products_df['Sector(s)'].str.contains('PreK-12').sum()
sector_dict = {'PreK-12':0,'Higher Ed':0, 'Corporate':0}

for sector in sector_dict.keys():
    val = products_df['Sector(s)'].str.contains(sector).sum()
    sector_dict.update({sector:val})


df4 = pd.DataFrame(columns=['Count'],index= sector_dict.keys())
df4.Count = sector_dict.values()
### plot pie 
fig = px.pie(df4, values='Count', names=df4.index, title='Sector(s) Distribution',color_discrete_sequence=["#ff228a","#20b1fd","#ffb703"])
fig.show()



__Observation:__
<div class="alert alert-block alert-warning">  
 <b>Prek-12</b>Which is from kindergarten to 12th grade, is dominant with 54%<br>
 <b>Higher Education</b>comes 2nd with ~28%<br>
 <b>Corprate</b>comes at the end with 18%<br>
</div>    


<div class="alert alert-block alert-success" align="center">  

   <h2>Prodcut - Primary Essential Function, Main & Sub-Categories</h2>
    
</div>  

In [ ]:
primary_essential_main = []
primary_essential_sub = []
for s in products_df["Primary Essential Function"]:
    if(not pd.isnull(s)):
        s1 = s.split("-",1)[0].strip()
        primary_essential_main.append(s1)
    else:
        primary_essential_main.append(np.nan)
    
    if(not pd.isnull(s)):
        s2 = s.split("-",1)[1].strip()
        primary_essential_sub.append(s2)
    else:
        primary_essential_sub.append(np.nan)

products_df["primary_essential_main"] = primary_essential_main
products_df["primary_essential_sub"] = primary_essential_sub

c1=c2=c3=0

for s in products_df["primary_essential_main"]:
    if(not pd.isnull(s)):
        c1 += s.count("CM")
        c2 += s.count("LC")
        c3 += s.count("SDO")
        
df5 = pd.DataFrame(columns=['Count'],index= ['CM','LC','SDO'])
df5.Count = [c1,c2, c3]
### plot pie 
fig = px.pie(df5, values='Count', names=df5.index, title='Primary Essential Function - Main',color_discrete_sequence=["#18ff9f","#2cfbff","#ffb703"])
fig.show()

In [ ]:
gc.collect()

__Observation:__
<div class="alert alert-block alert-warning">  
 <b>LC: Learning and Curriculum</b> comes 1st with 75% as the Primary Essential function of the Product<br>
 <b>CM: Classroom Management</b> comes 2nd with 13%<br>
 <b>SDO: School and District Operation</b> comes 3rd with 12%%<br>
</div>    


In [ ]:
lst = products_df["primary_essential_sub"].value_counts()
vals = products_df["primary_essential_sub"].dropna()
df6 = pd.DataFrame(columns=['Count'],index= vals)
df6.Count = lst
df6 = df6.sort_values(by='Count', ascending=True)
### plot bar 
fig = px.scatter(df6, x ="Count" , y =df6.index , title="Primary Essential Function - Sub-Categories", size='Count',color="Count",height=1000)
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
gc.collect()

__Observation:__
<div class="alert alert-block alert-warning">  
 <b>Digital Learning Platforms</b> wins it with about 74, which means they are the most engaged resource during 2020<br>
 <b>At the bottom we find:</b> <br>
    - Study Tools - tutoring, and <br>
    - Environmental, Health and Safety(EHS) Compliance<br>
</div>    


In [ ]:
sunb = products_df[['primary_essential_main','Provider/Company Name','Sector(s)']].dropna()
sunb = sunb.groupby(['primary_essential_main','Provider/Company Name']).size().reset_index(name = 'count')

fig = px.sunburst(sunb,path = ['primary_essential_main','Provider/Company Name'], values='count')
fig.show()


In [ ]:
sunb = products_df[['primary_essential_sub','Provider/Company Name','Sector(s)']].dropna()
sunb = sunb.groupby(['primary_essential_sub','Provider/Company Name']).size().reset_index(name = 'count')

fig = px.sunburst(sunb,path = ['primary_essential_sub','Provider/Company Name'], values='count',maxdepth=5)
fig.show()


In [ ]:
gc.collect()

<div class="alert alert-block alert-success" align="center">  

  <h2>Students Identified as Black / Hispanic in the districts; By State & Locale</h2>
    
</div>  

In [ ]:
districts_df["pct_black/hispanic"].replace({'[0, 0.2[':'0-20% B/H','[0.2, 0.4[':'20-40% B/H','[0.4, 0.6[':'40-60% B/H','[0.6, 0.8[':'60-80% B/H','[0.8, 1[':'80-100% B/H'},inplace=True)
sizes = districts_df["pct_black/hispanic"].value_counts()
labels = districts_df["pct_black/hispanic"].dropna().unique()
df7 = pd.DataFrame(columns=['Count'],index= labels)
df7.Count = sizes

### plot pie 
fig = px.pie(df7, values='Count', names=df7.index, title='Pourcentage of Black/Hispanic')
fig.show()

__Observation:__
<div class="alert alert-block alert-warning">  
 <b>About 19% to 32%</b> are identified as Black/Hispanic <br>
 
</div>    


In [ ]:
state_bh = districts_df.groupby("state")["pct_black/hispanic"].value_counts().to_frame().rename(columns={"pct_black/hispanic": "% of B/H"}).reset_index()

fig = px.treemap(state_bh, path=['state', 'pct_black/hispanic'], values='% of B/H',color='% of B/H', 
                 color_continuous_scale='Blackbody',
                title=('Pourcentage of Black/Hispanic Students by State'))
fig.update_layout(title_x=0.5)
                
fig.show()

In [ ]:
locale_bh = districts_df.groupby("locale")["pct_black/hispanic"].value_counts().to_frame().rename(columns={"pct_black/hispanic": "% of B/H"}).reset_index()

fig = px.treemap(locale_bh, path=['locale', 'pct_black/hispanic'], values='% of B/H',color='% of B/H', 
                 color_continuous_scale='Blackbody',
                title=('Pourcentage of Black/Hispanic Students by Locale'))
fig.update_layout(title_x=0.5)
                
fig.show()

<div class="alert alert-block alert-success" align="center">  

   <h2>Percentage of students in the districts eligible for Free / Reduced - Price of lunch; By State & Locale</h2>
    
</div>  

In [ ]:
districts_df["pct_free/reduced"].replace({'[0, 0.2[':'0-20% F/R','[0.2, 0.4[':'20-40% F/R','[0.4, 0.6[':'40-60% F/R','[0.6, 0.8[':'60-80% F/R','[0.8, 1[':'80-100% F/R'},inplace=True)
sizes = districts_df["pct_free/reduced"].value_counts()
labels = districts_df["pct_free/reduced"].dropna().unique()
df8 = pd.DataFrame(columns=['Count'],index= labels)
df8.Count = sizes

### plot pie 

fig = px.pie(df8, values='Count', names=df8.index, title='Pourcentage of Free/Reduced')
fig.show()

__Observation:__
<div class="alert alert-block alert-warning">  
 <b>from 35% till 67%</b> of students are eligible to Free/Reduced Price lunch meal.<br>
 
</div>    


In [ ]:
state_fr = districts_df.groupby("state")["pct_free/reduced"].value_counts().to_frame().rename(columns={"pct_free/reduced": "% of Free/Reduced"}).reset_index()

fig = px.treemap(state_fr, path=['state', 'pct_free/reduced'], values='% of Free/Reduced',color='% of Free/Reduced', 
                 color_continuous_scale='Greens',
                title=('Pourcentage of Free/Reduced Fees by State'))
fig.update_layout(title_x=0.5)
                
fig.show()

In [ ]:
locale_fr = districts_df.groupby("locale")["pct_free/reduced"].value_counts().to_frame().rename(columns={"pct_free/reduced": "% of Free/Reduced"}).reset_index()

fig = px.treemap(locale_fr, path=['locale', 'pct_free/reduced'], values='% of Free/Reduced',color='% of Free/Reduced', 
                 color_continuous_scale='Greens',
                title=('Pourcentage of Free/Reduced Fees by Locale'))
fig.update_layout(title_x=0.5)
                
fig.show()

In [ ]:
gc.collect()

<div class="alert alert-block alert-success" align="center">  

 <h2>Geographical Interpretation of Districts in State/Locale</h2>
    
</div>  

In [ ]:
def replace_BH_pct(range_str):
    if range_str == '0-20% B/H':
        return 0.1
    elif range_str == '20-40% B/H':
        return 0.3
    elif range_str == '40-60% B/H':
        return 0.5
    elif range_str == '60-80% B/H':
        return 0.7
    elif range_str == '80-100% B/H':
        return 0.9
    else:
        return np.nan
def replace_FR_pct(range_str):
    if range_str == '0-20% F/R':
        return 0.1
    elif range_str == '20-40% F/R':
        return 0.3
    elif range_str == '40-60% F/R':
        return 0.5
    elif range_str == '60-80% F/R':
        return 0.7
    elif range_str == '80-100% F/R':
        return 0.9
    else:
        return np.nan
    
def replace_ranges_raw(range_str):
    if range_str == '[4000, 6000[':
        return 5000
    elif range_str == '[6000, 8000[':
        return 7000
    elif range_str == '[8000, 10000[':
        return 9000
    elif range_str == '[10000, 12000[':
        return 11000
    elif range_str ==  '[12000, 14000[':
        return 13000
    elif range_str ==  '[14000, 16000[':
        return 15000
    elif range_str == '[16000, 18000[':
        return 17000
    elif range_str ==  '[18000, 20000[':
        return 19000
    elif range_str ==  '[20000, 22000[':
        return 21000
    elif range_str ==  '[22000, 24000[':
        return 21000
    else: 
        return np.nan
    
districts_df['pct_black_hispanic_mean'] = districts_df['pct_black/hispanic'].apply(lambda x: replace_BH_pct(x))
districts_df['pct_free_reduced_mean'] = districts_df['pct_free/reduced'].apply(lambda x: replace_FR_pct(x))
districts_df['pp_total_raw_mean'] = districts_df['pp_total_raw'].apply(lambda x: replace_ranges_raw(x))

In [ ]:
def plot_state_mean_for_var(col):
    temp = districts_df.groupby('states_abreviation')[col].mean().to_frame().reset_index(drop=False)

    fig = go.Figure()
    layout = dict(
        title_text = f"Mean {col} per State",
        geo_scope='usa',
    )

    fig.add_trace(
        go.Choropleth(
            locations=temp.states_abreviation,
            zmax=1,
            z = temp[col],
            locationmode = 'USA-states', # set of locations match entries in `locations`
            marker_line_color='white',
            geo='geo',
            colorscale="YlGnBu", 
        )
    )

    fig.update_layout(layout)   
    fig.show()

In [ ]:
states_abreviation = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District Of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

districts_df['states_abreviation'] = districts_df['state'].map(states_abreviation)

fig = go.Figure()
layout = dict(
    title_text = "The count of Districts in each State.",
    title_font = dict(
            family = "monospace",
            size = 25,
            color = "black"
            ),
    geo_scope = 'usa'
)

fig.add_trace(
    go.Choropleth(
        locations = districts_df['states_abreviation'].value_counts().to_frame().reset_index()['index'],
        zmax = 1,
        z = districts_df['states_abreviation'].value_counts().to_frame().reset_index()['states_abreviation'],
        locationmode = 'USA-states',
        marker_line_color = 'white',
        geo = 'geo',
        colorscale = "YlGnBu", 
    )
)
            
fig.update_layout(layout)   
fig.show()


In [ ]:
gc.collect()

In [ ]:
plot_state_mean_for_var('pct_black_hispanic_mean')
plot_state_mean_for_var('pct_free_reduced_mean')
plot_state_mean_for_var('pp_total_raw_mean')

In [ ]:
del df1, df_1, df1_1, df2, state_by_locale, df3__, df3, df3_, df4, df5, df6, sunb, df7, state_bh, locale_bh, df8, state_fr,locale_fr

In [ ]:
gc.collect()

# Initial Observations:

* There are a total of __233 School Districts__ available within the data, all around __USA__. 
    
    _📜 Def:_ A school district is a special-purpose district that operates local public primary and secondary schools in various nations.

* There are a total of __372 distinct Educational Technology Products__, such as:
    
    * Tools like _Canva_, 
    * Educational Apps like _Duolingo_,
    * Reading sites like _Goodreads_, 
    * Social pages like _Facebook_.

* The data was collected between _01.01.2020 (a few months before the pandemic hit) until 31.12.2020_. 

* Summing up all data collected for all the district we end up with ~ __22.3M rows__, which in the Data Science World would be called Big Data.

* From the 22M datapoints, around 24% have missing values in the __engagement_index__ feature. Nevertheless, the __pct_access__ feature is fully available.

* Around half of the pupils have at least one page load on a product and on a given day.

* There are almost 168 page loads per 1k students on a given product and on a given day. This means activity close to none for some of the pupils.

# Merging files 📁 = 📁+📁+📁

In [ ]:
engagement_df['time'] = pd.to_datetime(engagement_df['time'])

In [ ]:
print(products_df["LP ID"].nunique())
print(engagement_df["lp_id"].nunique())

## 1. Merging files: Product_Engagment_data 📁 = Products📁 + Engagment📁

In [ ]:
# products_engagement_data = pd.merge(products_df, engagement_df, left_on='LP ID', right_on='lp_id')
# products_engagement_data.head()

In [ ]:
#del products_df
gc.collect()

In [ ]:
print(districts_df["district_id"].nunique())
print(engagement_df["district_id"].nunique())

## 2. Merging files: Districts_Engagment_data 📁 = Districts📁 + Engagment📁

In [ ]:
engagement_df["district_id"] = engagement_df["district_id"].astype(str).astype(int)
districts_engagement_data = pd.merge(districts_df, engagement_df, left_on='district_id', right_on='district_id')
districts_engagement_data.head()

In [ ]:
del engagement_df
gc.collect()

In [ ]:
#products_engagement_data_1 = reduce_mem_usage(products_engagement_data)

In [ ]:
#districts_engagement_data_1 = reduce_mem_usage(districts_engagement_data)

In [ ]:
gc.collect()

In [ ]:
!pip install pdpipe

In [ ]:
import pdpipe as pdp

In [ ]:
agg_engagement_data = districts_engagement_data.groupby(["state", "time"],as_index=False)["engagement_index"].sum().reset_index()
agg_engagement_data.head()

In [ ]:
# set the size of the geo bubble , credits for : https://www.kaggle.com/gvyshnya/state-policy-impact-on-digital-learning-platforms/notebook#Digital-Learning-Platform-Patterns
def set_size(value):
    '''
    Takes the numeric value of a parameter to visualize on a map (Plotly Geo-Scatter plot)
    Returns a number to indicate the size of a bubble for a country which numeric attribute value 
    was supplied as an input
    '''
    result = np.log(1+value/100)
    if result < 0:
        result = 0.001
    return result


In [ ]:
pipeline = pdp.PdPipeline([
    pdp.ApplyByCols('engagement_index', set_size, 'size', drop=False),
    pdp.MapColVals('state', states_abreviation)
])

agg_engagement_data = pipeline.apply(agg_engagement_data)

agg_engagement_data.fillna(0, inplace=True)

agg_engagement_data = agg_engagement_data.sort_values(by='time', ascending=True)

agg_engagement_data.info()
agg_engagement_data.tail()

In [ ]:
fig = px.scatter_geo(
    agg_engagement_data, locations="state", locationmode='USA-states',
    scope="usa",
    color="engagement_index", 
    size='size', hover_name="state", 
    range_color= [0, 100000], 
    projection="albers usa", animation_frame= pd.to_datetime(agg_engagement_data["time"]).dt.month, 
    title='Engagement Index: LC - Digital Learning Platforms over the Months', 
    )

fig.show()

In [ ]:
fig = px.scatter_geo(
    agg_engagement_data, locations="state", locationmode='USA-states',
    scope="usa",
    color="engagement_index", 
    size='size', hover_name="state", 
    range_color= [0, 100000], 
    projection="albers usa", animation_frame= pd.to_datetime(agg_engagement_data["time"]).dt.day, 
    title='Engagement Index: LC - Digital Learning Platforms over the Days', 
    )

fig.show()

In [ ]:
fig = px.scatter_geo(
    agg_engagement_data, locations="state", locationmode='USA-states',
    scope="usa",
    color="engagement_index", 
    size='size', hover_name="state", 
    range_color= [0, 100000], 
    projection="albers usa", animation_frame= pd.to_datetime(agg_engagement_data["time"]).dt.weekday, 
    title='Engagement Index: LC - Digital Learning Platforms over the Days of the Week', 
    )

fig.show()

In [ ]:
agg_engagement_data_pct = districts_engagement_data.groupby(["state", "time"],as_index=False)["pct_access"].sum().reset_index()
agg_engagement_data_pct.head()

In [ ]:
pipeline = pdp.PdPipeline([
    pdp.ApplyByCols('pct_access', set_size, 'size', drop=False),
    pdp.MapColVals('state', states_abreviation)
])

agg_engagement_data_pct = pipeline.apply(agg_engagement_data_pct)

agg_engagement_data_pct.fillna(0, inplace=True)

agg_engagement_data_pct = agg_engagement_data_pct.sort_values(by='time', ascending=True)

agg_engagement_data_pct.info()
agg_engagement_data_pct.tail()

In [ ]:
fig = px.scatter_geo(
    agg_engagement_data_pct, locations="state", locationmode='USA-states',
    scope="usa",
    color="pct_access", 
    size='size', hover_name="state", 
    range_color= [0, 100000], 
    projection="albers usa", animation_frame= pd.to_datetime(agg_engagement_data["time"]).dt.month, 
    title='% Access: LC - Digital Learning Platforms over the Months', 
    )

fig.show()

In [ ]:
fig = px.scatter_geo(
    agg_engagement_data_pct, locations="state", locationmode='USA-states',
    scope="usa",
    color="pct_access", 
    size='size', hover_name="state", 
    range_color= [0, 100000], 
    projection="albers usa", animation_frame= pd.to_datetime(agg_engagement_data["time"]).dt.day, 
    title='% Access: LC - Digital Learning Platforms over the Days of Month', 
    )

fig.show()

In [ ]:
fig = px.scatter_geo(
    agg_engagement_data_pct, locations="state", locationmode='USA-states',
    scope="usa",
    color="pct_access", 
    size='size', hover_name="state", 
    range_color= [0, 100000], 
    projection="albers usa", animation_frame= pd.to_datetime(agg_engagement_data["time"]).dt.weekday, 
    title='% Access: LC - Digital Learning Platforms over the Days of the Week', 
    )

fig.show()

In [ ]:
def time_series_plot(df,col1,col2,col3):
    max_list = df[[col1,col2]]\
        .groupby([col1])[col2].mean()\
        .sort_values(ascending=False).index[:5].tolist()

    df = df[df[col1].isin(max_list)]\
                    .reset_index(drop=True)[[col3, col1, col2]]
    
    df = df.pivot_table(index=col3, columns=col1, values=col2)

    fig = px.line(df, facet_col=col1, facet_col_wrap=1, width=900, height=1500)
    fig.update_layout(
                      title=("Top :"+ col1 + " , " + col2 + " , " + col3).title(),
                      title_x=0.39,
                      template="plotly_white",
                      paper_bgcolor='#f5f7f8',
                      font = {'family': 'Serif', 'size': 20}
                     )
    fig.show()


<div class="alert alert-block alert-success" align="center">  

 <h2>Districts Engagement : By State, % Access, Time</h2>
    
</div>  

In [ ]:
time_series_plot(districts_engagement_data,"state","pct_access","time")

__Observation:__
<div class="alert alert-block alert-warning">  
 <b>Like we see above that the % of Access in the state during Jun,Jul, Aug is low due to summer vacation and also in the late Dec due to holidays too</b> <br>
    The % varies from 0 to 6% as in Arizona as the highest during Aug 7th, 2020. 
    
    
    The low values are aligned with Holidays.
 
</div>    


<div class="alert alert-block alert-success" align="center">  

 <h2>Districts Engagement : By State, Engagement Index, Time</h2>
    
</div>  

In [ ]:
time_series_plot(districts_engagement_data,"state","engagement_index","time")


<div class="alert alert-block alert-success" align="center">  

 <h2>Districts Engagement : By Locale, % Access, Time</h2>
    
</div>  

In [ ]:
time_series_plot(districts_engagement_data,"locale","pct_access","time")

<div class="alert alert-block alert-success" align="center">  

 <h2>Districts Engagement : By Locale, Engagement Index, Time</h2>
    
</div>  

In [ ]:
time_series_plot(districts_engagement_data,"locale","engagement_index","time")

In [ ]:
geolocator = Nominatim(user_agent="Ruch")

def feature_generation(df):
    lat=[]
    long=[]
    for i in df['state']: 
        location = geolocator.geocode(i)
        try:
            lat.append(location.latitude)
            long.append(location.longitude)
        except:
            lat.append("NA")
            long.append("NA")
    df['Latitude'] = lat
    df['Longitude'] = long
    
    return df

def map_df(df,col1,col2):
    df = pd.DataFrame(df[[col1,col2]]\
            .groupby([col1])[col2].mean()\
            .sort_values(ascending=False)[:50]).reset_index()
    
    df = feature_generation(df)
    
    return df

state_access = map_df(districts_engagement_data, "state", "pct_access")
state_engagement_index = map_df(districts_engagement_data, "state", "engagement_index")

In [ ]:
north_america_map = folium.Map(location=[38.9, -77.05], tiles='Stamen Watercolor', zoom_start=3)
north_america_map

<div class="alert alert-block alert-success" align="center">  

 <h2>Map - Districts Engagement : By State, % Access</h2>
    
</div>  

In [ ]:
mc = MarkerCluster()
for idx, row in state_access.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        popup = """
        State : <b>%s</b><br>
        Percentage Access : <b>%s</b><br>
        """ % (row['state'], row['pct_access'])
        mc.add_child(Marker([row['Latitude'], row['Longitude']],tooltip=popup))
    north_america_map.add_child(mc)
north_america_map

<div class="alert alert-block alert-success" align="center">  

 <h2>Map - Districts Engagement : By State, Engagement Index</h2>
    
</div>  

In [ ]:
mc = MarkerCluster()
for idx, row in state_engagement_index.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        popup = """
        State : <b>%s</b><br>
        Engagement Index : <b>%s</b><br>
        """ % (row['state'], row['engagement_index'])
        mc.add_child(Marker([row['Latitude'], row['Longitude']],tooltip=popup))
    north_america_map.add_child(mc)
north_america_map

Engagement Day/Week/Month

In [ ]:
#since the year was the same 2021 we don't require the year column
districts_engagement_data['month'] = districts_engagement_data['time'].dt.month
districts_engagement_data['day']=districts_engagement_data['time'].dt.day
districts_engagement_data['week']=districts_engagement_data['time'].dt.weekday

In [ ]:
districts_engagement_data.head()

In [ ]:
data_agg=districts_engagement_data.groupby(['day','week','month']).agg({'engagement_index':np.mean,'pct_access':np.mean})
data_agg

In [ ]:
data_agg = data_agg.reset_index()
data_agg

In [ ]:
plt.figure(figsize=(15,11))
sns.lineplot(y=data_agg['pct_access'],x=data_agg["month"],palette='rocket')
plt.title("Average access per month")
plt.show()

In [ ]:
plt.figure(figsize=(15,11))
sns.lineplot(y=data_agg['engagement_index'],x=data_agg["month"],palette='rocket')
plt.title("Average Engagemnet Index per month")
plt.show()

In [ ]:
plt.figure(figsize=(15,11))
sns.lineplot(hue=data_agg['week'],y=data_agg['pct_access'],x=data_agg["month"],palette='rocket')
plt.title("Average access per month per day of the week 0=Monday, to , 6=Sunday")
plt.show()

In [ ]:
plt.figure(figsize=(15,11))
sns.lineplot(hue=data_agg['week'],y=data_agg['engagement_index'],x=data_agg["month"],palette='twilight_shifted')
plt.title("Average access per month per day of the week 0=Monday, to , 6=Sunday")
plt.show()

__Observation:__
<div class="alert alert-block alert-warning">  
     <b> Engagement & % Access is very high during first months and from August to Decemeber.</b> <br>
    <b>Like we see above that the % of Access & the Engagement Index during Jun,Jul, Aug is low due to summer vacation and also in the late Dec due to holidays too</b> <br>

 
</div>    


In [ ]:
plt.figure(figsize=(15,10))
sns.lineplot(y=data_agg['engagement_index'],x=data_agg["week"])
plt.title("Engagement Index over days of the week 0=Monday, to , 6=Sunday")
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.lineplot(y=data_agg['pct_access'],x=data_agg["week"])
plt.title("Percentage Access over days of the week 0=Monday, to , 6=Sunday")
plt.show()

__Observation:__
<div class="alert alert-block alert-warning">  
     <b> As expected there is more engagment/ % Access at the begining of the week and it slowly drops as we get to the weekend , with Tuesday having the highest number of average engagment</b> <br>

 
</div>    



In [ ]:
from wordcloud import WordCloud, STOPWORDS 

In [ ]:
cloud = WordCloud(width=1440, height=1080,stopwords={'nan'}).generate(" ".join(districts_df['state'].astype(str)))
plt.figure(figsize=(15, 10))
plt.imshow(cloud)
plt.axis('off')
plt.show()

In [ ]:
cloud = WordCloud(width=1440, height=1080,stopwords={'nan'}).generate(" ".join(products_df['Product Name'].astype(str)))
plt.figure(figsize=(15, 10))
plt.imshow(cloud)
plt.axis('off')
plt.show()

In [ ]:
cloud = WordCloud(width=1440, height=1080,stopwords={'nan'}).generate(" ".join(products_df['Provider/Company Name'].astype(str)))
plt.figure(figsize=(15, 10))
plt.imshow(cloud)
plt.axis('off')
plt.show()

# Reference 🙏


* **Plotly basic Charts** : https://plotly.com/python/basic-charts/



# ⏳ Work is still on Progress ... 

<div class="alert alert-block alert-danger" align="center">  

 <h2>Final Observations, Recommendations</h2>
    
</div>  

<div class="alert alert-block alert-success" align="left">  


    


<h1>Learning Districts</h1>

As general observations, most data was registered in states like Connecticut, Utah, Massachusetts, Illinois or California. More than 50% of the districts are located in the suburbs.

Looking at the wheel we can see the top 4 characteristics of each state - and whether or not the majority of the schools in the state:

* have more than 50% of students identified as Black/Hispanic;
* have more than 50% of students eligible for free or reduced price lunch;
* have good connection to internet or not (almost all observations had the same interval, so we won't look at this characteristic going further);
* have more than $1,000 per pupil expenditure (marked with deeper red, less than $1,000 marked with lighter red).


<h2>📅 Student Activity Through Time</h2>

* The median percentage of active users on a given day for a product varies widely, from New York having on average ~0.14% activity to states like California or Washington with only 0.1% active users per product on a given day.
* The Activity Trend is general for almost all states - with increased activity during the school year and dormant activity during holidays, like the Summer Break or towards the end of the year (Winter Recess).


<h2>📚 Student Engagement Through Time</h2>

The engagement's pattern looks very similar with the activity one:


* After the start of the pandemic and the official closing of schools for the remainer of the year, the median page loads per product has increased visibly for most of the states.
* Compared to the activity analyzed above, the engagement doesn't decrease so drastically during the summer holiday. This is because lots of Educational Products are used in the free time too.
* Although the second part of the year (when the School Year 2020-2021 started) most of the states opted for hybrid or from home study, there is a slight decrease in the median engagement per product per 1k students.


<h2>🇺🇸 States Engagement</h2>

* The biggest engagement registered in 2020 is in North Dakota and Arizona (however, we need to keep in mind that these states had data available from only one district).
* Connecticut and Massachusetts are the states with the biggest true engagement, based on the fact that there was data collected from more than 20 districts for each of these states.
* We can't say that a region in US is more active than another, as the available states are quite dispersed throughout the country.

<h2>🚸 Is there an Engagement Gap?</h2>

* On average, there is lower engagement in the states that have a majority of black or hispanic students vs the ones that have only a partial number (less than 50%). This trend is mantained ~equal throughout the entire year.
* There is a lower engagement in the states where the majority of students receive free or reduced lunch vs the states were less than 50% of them do. However, we can observe that this difference narrows in the second part of the year.
* A very interesting finding is that, although children in states with big expenditure per pupil have registered bigger engagement, there is no significantly big difference between them and states where the expenditure per pupil is much lower. Moreover, the difference narrows even more in the second half of the year.


<h1>Learning Products</h1>
    
<h2>🍎🍎🍎 Top Products Overview</h2>

* Most used educational products in 2020 were Google products (like Google Docs, Google Classroom, Google Drive, Google Forms, Google Sheets, Google Meet), Youtube and Kahoot!.
* Compared to these few products, all the others had extremely little to no median engagement throughout the year. Hence, Google and Kahoot! have monopol so far over the online educational instruments.

<h2>🎒🎒🎒 Learning Product Engagement Timeline</h2>

* Based on the most engaging 10 products, we can see that there is an increase in the new school year 2020-2021 vs the engagement before the big summer break. This means that students started not only getting used to these products, but also realised that they're here to stay and they need to start using them more often.

* Youtube started increasing in engagement during summer 2020 and sustained it's activity in the new school year.

* Although after the start of the pandemic there has been considerable increase in usage for these e-learning tools, the difference after vs the little time we have available before the pandemic isn't that substantial. This could happen because most of the kids that were active after the pandemic started were also active using these tools before. However, there is a big pool of students that seem to not use these products at all (and this can be seen in the activity percentage).

🏫🏫🏫 **Learning Product Engagement on States**

* Illinois, Indiana, New York, Connecticut and Ohio are the states with the biggest engagement in almost all 10 learning products.
* Utah, California, Florida, Tennessee, Texas, Michigan, Minnesota and Virginia are the states with the lowest engagement in almost all 10 learning products.


<h1>📑📑 Recommendation</h1>



* Activity and Engagement increased a bit after the start of the pandemic and then slightly decreased in the beginning of the academic year 2020-2021 (however, this trend isn't shown in top 10 products). This means that both teachers and students started using more diligently a few products that quickly monopolized the others, however the overall average engagement is low and alarming, at least compared to what I would have expected to see.
* We learned that the states where there are bigger percentages of black/hispanic students or more pupils that benefit of free/reduced lunch need better care and attention. North Dakota, Arizona, New York, New Hempshire and Connecticut are the states that stand out in terms of the biggest engagement in comparison to the others. However, the overall low engagement may be caused by:
    * **Lack of age appropiate skills**: children in Primary and Middle school don't know yet how to properly use the technology and might get stuck and frustrated on the way. If the parent or tutor doesn't know, can't or isn't able to be there for them when they need it, they might fall short and stop using the educational products alltogether.
    * **Lack of technology resources**: families that have 2 or more children would need to use a distinct laptop/tablet in the same time during the online courses AND have a proper sepparate (quiet) space to learn. Unfortunately, many families lack these kind of resources.
    * **Low Emotional Stability**: truth is some children started getting tired of the "new norm". Despite the emotional support they have from their teacher, lack of interaction might have accumulated, leading to them not wanting to engage and participate during the online classes and/or during homework.
    * **Low energy**: Staying in from of a computer 6 hours a day and then doing the homework (also digitaly) for another 2 to 3 hours can take a toll on children's overall wellbeing. This accumulates to 8+ hours of staying in a chair, 5 times a week, equivalent to a full time job.

* There have been active efforts from teachers **to improve their technological skills as well as overall curriculum**, in an effort to adapt it as much as possible for the "online" classrooms. Moreover, they are active in keeping in touch and reaching out more to parents and children. These acts are extremely benefficial, narrowing the communication gap that has been inevitably formed while staying at home.


<h1>📑📑📑 Suggestions</h1>


1. Keeping schools open as much as possible should be a priority, as this solves many problems that the pandemic brought to begin with. To keep the safety, I would move into online format for 2 weeks only classes that have a confirmed case, ~10 days if no new case has been found between the other students.

2. For online classes, teachers should try and brainstorm as many new, fun, out of the ordinary lectures to keep the students engaged. This way, they will join every day with a curiosity in mind: "What will we do today?". New games, movies or fun interactive ways to teach might encourage them to pay attention, despite being alone at home and not in a classroom full of people.

3. Better marketing for e-learning products. As we saw, there are only 6-7 products that ar highly used. This is because teachers do not know about other apps and do not have the necessary time to research. Hence, I would designate a new job (part time or full time), where the requirements would be to always be up to date with e-learning tools and to recommend and explain to teachers what and how to use them. Moreover, I would make this customized on a grade and lecture basis: a 1st grade teacher will need different e-learning tools vs. what a math teacher in 8th grade might need. Teachers already have their hands full, so a designated person that will share customized tools for everybody will help grow both the engagement of the students (bringing something new and exciting to the table) and the engagement of the product.

4. The e-learning tools should always be FREE for the end-user: either free by default or paid by the educational institute. Assuming that the teacher or parent can afford any additional subscription for e e-learning tool would be faulty and unrealistic.

5. We should not underestimate the power of these e-learning tools. Even after the pandemic is over, teachers could still use these amazing well thought apps to show young future adults how to navigate through technology and to learn in an interactive different way. Helping the low engagement products gain visibility will benefit the pupils, teachers and overall market tremendously.

6. Teachers should take part in IT courses, so they could get better acquinted with the digital world.

7. Online classes should be shorter and have different curriculum (and engaging, as stated at 2.). Teachers should also coordinate their assignments every week, so pupils would not have an overload of work at once. Parents should be involved as well, to be sure that not only children take part in all their online classes, but that they also have some time appart from the devices. It would be a shame to shorten the time spent at the computer for online classes, but the child to immediately start a game or browse social media afterwards for hours on end. Encouraging them to get outside in a safe manner will benefit their Social and Emotional anxiety.

8. Introduce more psychologists in schools. I would also implement a mandatory 1to1 for each student and psychologist at least every few weeks. This way we can check in as adults and catch early any metal health issue or distress that the child might develop.

9. A big issue I struggle to find a fix to is for smaller children. School's main purpose for Primary pupils is to involve and integrate them into the society, hence any online classes will struggle to meet this goal. Furthermore, little children will need mandatory help and guidance from their parents/tutors using the technology. Hence, I would encourage smaller sizes of classes and a lighter policy whenever any Covid-19 case is found, as their tolerance to the virus is very good.
    
    
</div>  


<div class="alert alert-block alert-info" align="center">
    <h1>🙏 Thanks to anyone who UP voted this Notebook 🙏</h1>
</div>